<a href="https://colab.research.google.com/github/pinilDissanayaka/Fine-Tuning-Pretrained-DistilBERT-sinhala-sentiment-analysis/blob/main/Fine_Tuning_Pretrained_DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install datasets
! pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
pip install transformers

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.preprocessing import LabelEncoder
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.metrics import classification_report, confusion_matrix
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.functional import softmax

In [5]:
huggingface_dataset=load_dataset("sinhala-nlp/sinhala-sentiment-analysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.tsv:   0%|          | 0.00/35.8M [00:00<?, ?B/s]

test.tsv:   0%|          | 0.00/8.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7229 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1819 [00:00<?, ? examples/s]

In [6]:
huggingface_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'article_id', 'title', 'author', 'date', 'body', 'comment_author', 'comment_date', 'comment_phrase', 'comment_sentiment'],
        num_rows: 7229
    })
    test: Dataset({
        features: ['id', 'article_id', 'title', 'author', 'date', 'body', 'comment_author', 'comment_date', 'comment_phrase', 'comment_sentiment'],
        num_rows: 1819
    })
})

In [7]:
huggingface_dataset["train"]

Dataset({
    features: ['id', 'article_id', 'title', 'author', 'date', 'body', 'comment_author', 'comment_date', 'comment_phrase', 'comment_sentiment'],
    num_rows: 7229
})

In [8]:
x_train=list(huggingface_dataset["train"]['comment_phrase'])
y_train=list(huggingface_dataset["train"]['comment_sentiment'])

In [9]:
x_test=list(huggingface_dataset["test"]['comment_phrase'])
y_test=list(huggingface_dataset["test"]['comment_sentiment'])

In [17]:
print(f"x_train: {len(x_train)}")
print(f"y_train: {len(y_train)}")
print(f"x_test: {len(x_test)}")
print(f"y_test: {len(y_test)}")

x_train: 7229
y_train: 7229
x_test: 1819
y_test: 1819


In [18]:
np.unique(y_train)

array([0, 1, 2])

In [19]:
label_encoder=LabelEncoder()
y_train=label_encoder.fit_transform(y_train)
y_test=label_encoder.transform(y_test)

In [20]:
encorded_labels=list(np.unique(y_train))

decoded_labels=label_encoder.inverse_transform(encorded_labels)


for encoded_label, decoded_label in zip(encorded_labels, decoded_labels):
    print(f"Encoded Label: {encoded_label} --> Decoded Label: {decoded_label}")


num_labels=len(encorded_labels)

print(f"num_labels: {num_labels}")

Encoded Label: 0 --> Decoded Label: 0
Encoded Label: 1 --> Decoded Label: 1
Encoded Label: 2 --> Decoded Label: 2
num_labels: 3


In [21]:
class MyDataset(Dataset):
    def __init__(self, text, labels, tokenizer):
        self.text = text
        self.labels=labels
        self.tokenizer = tokenizer


        self.dataset={}

        text_tokens = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        self.dataset["input_ids"] = torch.tensor(text_tokens["input_ids"])
        self.dataset["attention_mask"] = torch.tensor(text_tokens["attention_mask"])
        self.dataset["labels"] = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.dataset["input_ids"][idx],
            "attention_mask": self.dataset["attention_mask"][idx],
            "labels": self.dataset["labels"][idx]}

    def __len__(self):
        return len(self.labels)

In [22]:
d_bert_tokenizer=DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [24]:
train_data_loader=DataLoader(MyDataset(x_train, y_train, d_bert_tokenizer), batch_size=16, shuffle=True)
test_data_loader=DataLoader(MyDataset(x_test, y_test, d_bert_tokenizer), batch_size=16, shuffle=True)

<ipython-input-21-484d388013c9>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.dataset["input_ids"] = torch.tensor(text_tokens["input_ids"])
<ipython-input-21-484d388013c9>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.dataset["attention_mask"] = torch.tensor(text_tokens["attention_mask"])


In [25]:
model=DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
epochs=10

optimizer = Adam(model.parameters(), lr=2e-5)

for epoch in range(epochs):
    for batch in train_data_loader:
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        optimizer.zero_grad()

        input_ids=batch["input_ids"].to(device)
        attention_mask=batch["attention_mask"].to(device)
        labels=batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

Epoch 1/10, Loss: 1.0811126232147217
Epoch 1/10, Loss: 1.1392555236816406
Epoch 1/10, Loss: 1.0943492650985718
Epoch 1/10, Loss: 1.1037356853485107
Epoch 1/10, Loss: 1.1008679866790771
Epoch 1/10, Loss: 1.0853475332260132
Epoch 1/10, Loss: 1.1222337484359741
Epoch 1/10, Loss: 1.0859203338623047
Epoch 1/10, Loss: 1.1084873676300049
Epoch 1/10, Loss: 1.0831501483917236
Epoch 1/10, Loss: 1.0867023468017578
Epoch 1/10, Loss: 1.1151164770126343
Epoch 1/10, Loss: 1.0789375305175781
Epoch 1/10, Loss: 1.0943924188613892
Epoch 1/10, Loss: 1.0452256202697754
Epoch 1/10, Loss: 1.0738874673843384
Epoch 1/10, Loss: 1.031264066696167
Epoch 1/10, Loss: 1.0589492321014404
Epoch 1/10, Loss: 1.0162206888198853
Epoch 1/10, Loss: 0.9556481838226318
Epoch 1/10, Loss: 0.9292578101158142
Epoch 1/10, Loss: 1.1470582485198975
Epoch 1/10, Loss: 0.8833310008049011
Epoch 1/10, Loss: 0.9606323838233948
Epoch 1/10, Loss: 1.1278873682022095
Epoch 1/10, Loss: 1.0670100450515747
Epoch 1/10, Loss: 1.0235638618469238
Ep